In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/2021_NLP_Project

/content/drive/MyDrive/Colab Notebooks/2021_NLP_Project


In [ ]:
max_len_value = 64
bert_model_name = 'bert-base-multilingual-cased'
train_test_split_ratio = 0.1
train_val_split_ratio = 0.1
batch_size_value = 32
learning_rate = 2e-5
epochs_value = 4

In [ ]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 2.8 MB 11.1 MB/s 
     |████████████████████████████████| 3.3 MB 41.4 MB/s 
     |████████████████████████████████| 895 kB 35.8 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 636 kB 46.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

import pickle
from tqdm import tqdm

# from transformers import RobertaTokenizer, RobertaModel
# from transformers import DistilBertTokenizer, DistilBertModel

In [ ]:
# 파일 로드
with open('./data/final_idiom_dataset_for_ko.pkl', 'rb') as f:
  corpus = pickle.load(f)

In [ ]:
# TroFi dataset을 불러온다.
corpus.head(10)

,ko,en,Label
0,다만 지난 7월부터 수면 무호흡증 진단을 위한 수면다원검사와 치료에 필요한 양압기 ...,"However, since July, as health insurance (20% ...",0
1,하지만 몇몇 사람들은 그 표현이 관용구인지 알아차리지 못합니다.,"Some people, however, cant recognize that it w...",0
2,끈끈한 승부근성으로 찬스에서 더 영양가 만점의 활약을 하는 오재원이 있기에 선두질주...,The Doosan Bears are gaining momentum in the l...,1
3,협약내용은 삼성화재서비스손해사정㈜에서 매월 임직원들의 기부를 통해 향후 3년간 30...,The contents of the agreement are to support m...,0
4,이 같은 대중적 관심을 겨냥해 이동통신사가 발 빠르게 움직였다.,"In response to such public interest, mobile op...",1
5,"달러보험은 원화상품 대비 달러가 갖는 경쟁력을 바탕으로 고객가치를 높인 상품으로, ...",The dollar insurance is a product that heighte...,0
6,극 중 커플로 등장했던 박경림과 조인성이 사석에서 술잔을 기울이 박경림의 절친인 장...,"Park Kyung-rim and Jo In-sung, who appeared as...",1
7,"제1 야당이 극우 인사의 해괴망측한 주장에 멍석을 깔아주고, 함께 장단을 맞췄는데 ...",The main opposition party dug up the weird cla...,1
8,금융규제에 발이 묶여 시장 진입 자체가 힘든 상황이 지속되자 국회와 정부 부처가 모...,As financial regulations continue to make it d...,1
9,1년 이상의 파트 타임 경험이 있습니다.,I have an experience of part time job longer t...,0


In [ ]:
text = corpus['ko']
label = corpus['Label']
n_topic = len(set(label))

In [ ]:
label.sum(), label.sum()/len(label)

(3376, 0.5)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(text, label, test_size=train_test_split_ratio)

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir='bert_eng_ckpt', do_lower_case=False)

훈련셋

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(tex) + " [SEP]" for tex in x_train]
sentences[:10]

['[CLS] 김경진 민주평화당 의원은 존리 대표에게 “세계적 기업으로 국내 매출액을 밝히지 못하는 게 부끄럽지 않냐”고 날을 세웠다. [SEP]',
 '[CLS] 신동욱 공화당 총재가 남북관계 진전을 연인에 빗대 논란을 빚은 민병두 더불어민주당 의원을 두고 “X버릇 몸에 밴 골”이라고 비판했다. [SEP]',
 '[CLS] 브리티시텔레콤의 후신인 BT 관계자는 자신들이 현행 계획법을 준수하고 있으며, 경우에 따라서는 자치의회가 원하지 않으면 부스 철거를 동의한다고 밝혔다. [SEP]',
 '[CLS] 물만 사면 되는데 근처에 마트라도 찾을 수 있을까요? [SEP]',
 '[CLS] 어제 눈에 불을 켜고 찾아봤거든요. [SEP]',
 '[CLS] 전화는 먼저 하지 않겠다고 말하면서도 오매불망 재형이 오기만을 손꼽아 기다리는 박세완의 모습은 안타까운데 귀여움이 폭발했다. [SEP]',
 '[CLS] 법원은 지난 21일 “증거가 확보돼 있고 수사에 임하는 태도 등을 볼 때 구속의 필요성을 인정하기 어렵다"며 김 지회장의 영장을 기각했다. [SEP]',
 '[CLS] 저는 연구개발부에서 일해요. [SEP]',
 '[CLS] 삼성이 이재용 삼성전자 부회장의 경영권 승계를 위해 제일모직·삼성물산 합병을 추진하면서 유령사업으로 기업가치를 부풀렸다는 주장이 나왔다. [SEP]',
 '[CLS] 당신이 적정한 타입의 밸브를 선정해서 견적서를 보내 주세요. [SEP]']

In [ ]:
# 라벨 추출
labels = y_train.values
labels

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained(bert_model_name, cache_dir='bert_kor_ckpt', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

[CLS] 김경진 민주평화당 의원은 존리 대표에게 “세계적 기업으로 국내 매출액을 밝히지 못하는 게 부끄럽지 않냐”고 날을 세웠다. [SEP]
['[CLS]', '김', '##경', '##진', '민', '##주', '##평', '##화', '##당', '의', '##원은', '존', '##리', '대', '##표', '##에게', '[UNK]', '세계', '##적', '기', '##업', '##으로', '국', '##내', '매', '##출', '##액', '##을', '밝', '##히', '##지', '못', '##하는', '게', '부', '##끄', '##럽', '##지', '않', '##냐', '[UNK]', '고', '날', '##을', '세', '##웠다', '.', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = max_len_value

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8935,  31720,  18623,   9311,  16323, 119398,  18227,
        21928,   9637,  84467,   9680,  12692,   9069,  37824,  26212,
          100,  32613,  14801,   8932,  26784,  11467,   8909,  31605,
         9258,  52363, 119122,  10622,   9324,  18108,  12508,   9290,
        12178,   8872,   9365, 118702, 118867,  12508,   9523, 118728,
          100,   8888,   8985,  10622,   9435, 108602,    119,    102,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=train_val_split_ratio)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=train_val_split_ratio)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,  39671,  10273,  21789,  38851,  41886,   9450,  51431,  10622,
          9256, 119277,  37093,   9992,  37004,  12310,   8909,  32537,  20626,
         16605,  31503,  55635,   9365, 119270, 100434,   9706,  16758,  13441,
         18347,    117,  66402,  26784,  21611,   8900,  21611,  13764,  72482,
         10459,   9580, 119249,    113,  17461,  42784,    114,    117,   9694,
        118634,  12310,  26784,  12030,  25486,  33188,  14863,   9706,  12092,
         14646,   9248, 119249,    113,  16367,  43494,    114,    117,   8908,
         22458])
tensor(1)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([   101, 103988,  14040,  12638,   9770,  43852,  11287,   9977,  25387,
         12092,  14040,   9625,  18622,  105

In [ ]:
# 배치 사이즈
batch_size = batch_size_value

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

테스트셋

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in x_test]
# sentences[:10]

In [ ]:
# 라벨 추출
labels = y_test.values
# labels

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=False)
# tokenizer = RobertaTokenizer.from_pretrained(bert_model_name, do_lower_case=False)
# tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name, do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

[CLS] 패스트트랙 추진을 놓고 여야 4당과 자유한국당의 극한대치가 이어지면서 반말과 고성을 주고받는 촌극도 연일 계속돼 대중들의 눈살을 찌푸리게 하고 있다. [SEP]
['[CLS]', '패', '##스트', '##트', '##랙', '추', '##진', '##을', '놓', '##고', '여', '##야', '4', '##당', '##과', '자', '##유', '##한', '##국', '##당', '##의', '극', '##한', '##대', '##치가', '이어', '##지', '##면서', '반', '##말', '##과', '고', '##성을', '주', '##고', '##받', '##는', '촌', '##극', '##도', '연', '##일', '계속', '##돼', '대', '##중', '##들의', '눈', '##살', '##을', '찌', '##푸', '##리', '##게', '하고', '있다', '.', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = max_len_value

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9909,  34994,  15184, 118855,   9765,  18623,  10622,
         9029,  11664,   9565,  21711,    125,  21928,  11882,   9651,
        42815,  11102,  20479,  21928,  10459,   8925,  11102,  14423,
       104504,  64749,  12508,  30936,   9321,  89523,  11882,   8888,
        36456,   9689,  11664, 118965,  11018,   9759,  63243,  12092,
         9568,  18392,  77039, 118798,   9069,  41693,  25258,   9034,
       106249,  10622,   9727, 119405,  12692,  14153,  32487,  11506,
          119,    102,      0,      0,      0,      0,      0,      0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9909,  34994,  15184, 118855,   9765,  18623,  10622,   9029,
         11664,   9565,  21711,    125,  21928,  11882,   9651,  42815,  11102,
         20479,  21928,  10459,   8925,  11102,  14423, 104504,  64749,  12508,
         30936,   9321,  89523,  11882,   8888,  36456,   9689,  11664, 118965,
         11018,   9759,  63243,  12092,   9568,  18392,  77039, 118798,   9069,
         41693,  25258,   9034, 106249,  10622,   9727, 119405,  12692,  14153,
         32487,  11506,    119,    102,      0,      0,      0,      0,      0,
             0])
tensor(1)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0.])


In [ ]:
# 배치 사이즈
batch_size = batch_size_value

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

모델 생성

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=2)
# model = RobertaModel.from_pretrained(bert_model_name, num_labels=2)
# model = DistilBertModel.from_pretrained(bert_model_name, num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = learning_rate, # 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = epochs_value

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

모델 학습

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    accuracy = np.sum(pred_flat == labels_flat) / len(labels_flat)
    precision = precision_score(pred_flat, labels_flat)
    recall = recall_score(pred_flat, labels_flat)
    f1score = f1_score(pred_flat, labels_flat)
    return accuracy, precision, recall, f1score

    # return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        # outputs = model(**b_input_ids, 
        #                 token_type_ids=None, 
        #                 attention_mask=b_input_mask, 
        #                 labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
            # outputs = model(**b_input_ids, 
            #                 token_type_ids=None, 
            #                 attention_mask=b_input_mask, 
            #                 labels=b_labels)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        # tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        tmp_eval_accuracy, _, _, _ = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.41
  Training epcoh took: 0:02:02

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:04

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.18
  Training epcoh took: 0:02:02

Running Validation...
  Accuracy: 0.95
  Validation took: 0:00:04

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.11
  Training epcoh took: 0:02:02

Running Validation...
  Accuracy: 0.95
  Validation took: 0:00:04

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.07
  Training epcoh took: 0:02:01

Running Validation...
  Accuracy: 0.96
  Validation took: 0:00:04

Training complete!


In [ ]:
torch.save(model, PATH)

테스트셋 평가

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
# eval_loss, eval_accuracy = 0, 0
eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1score = 0, 0, 0, 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        # outputs = model(**b_input_ids, 
        #                 token_type_ids=None, 
        #                 attention_mask=b_input_mask, 
        #                 labels=b_labels)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    # tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    tmp_eval_accuracy, tmp_eval_precision, tmp_eval_recall, tmp_eval_f1score = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy

    eval_precision += tmp_eval_precision
    eval_recall += tmp_eval_recall
    eval_f1score += tmp_eval_f1score

    nb_eval_steps += 1

print("")
# print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Precision: {0:.3f}".format(eval_precision/nb_eval_steps))
print("Recall: {0:.3f}".format(eval_recall/nb_eval_steps))
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
print("f1_score: {0:.3f}".format(eval_f1score/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Precision: 0.973
Recall: 0.889
Accuracy: 0.928
f1_score: 0.925
Test took: 0:00:05


새로운 문장 테스트

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = max_len_value

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        # outputs = model(**b_input_ids, 
        #                 token_type_ids=None, 
        #                 attention_mask=b_input_mask, 
        #                 labels=b_labels)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
logits = test_sentences([''])

print(logits)
print(np.argmax(logits))

In [ ]:
logits = test_sentences(['I could not find anything particularly weird.'])

print(logits)
print(np.argmax(logits))

In [ ]:
logits = test_sentences(['The distrust that Korea\'s policy may change when the administration of Korea changes has been settled in Japan, so Japan Merely Looks On regarding the progress of the Korea-Japan relations.'])

print(logits)
print(np.argmax(logits))